# **INITIALIZATION**

Import libraries.

In [39]:
# Library for reading and writing data to and from files
import os
# Library for numerical computing
import numpy as np
# Library for mathematical functions
import math
# Library for getting dictionaries from data structures
from collections import defaultdict
# Library to get fit models
from scipy.optimize import curve_fit
# Library to plot histograms
import matplotlib.pyplot as plt

Define paths.

In [40]:
# Dataset directory
corpus_directory='../Dataset/corpus2mw'

Set the seed for reproducibility.

In [41]:
np.random.seed(55555555)

# **GET TOKENS**

First, define the special characters to be separated from each word.

In [42]:
# List of characters
specialchars = [';', ':', '!', '?', '<', '>', '&', ')', '(', ']', '[', ',', '.', '"', '%', '$ ', '=', '}', '{', '-', '_', '+', '*', '#', '@']

And define the function which separates the special characters from each word, assuming they can only be before or after each word.

In [43]:
def token(w):
    # Init the empty list of tokens
    res = []

    # If the length is 1, add the character whatever it is
    if len(w) == 1:
        res.append(w)
    
    # Otherwise (if it's at least two characters)
    else:
        # If the first character is special, add it to the list and remove it from the word
        if w[0] in specialchars:
            res.append(w[0])
            w = w[1:]
        
        # Now, if the length became 1 because of that, for the same reason as before, add the character whatever it is
        if len(w) == 1:
            res.append(w)
        # Otherwise (if it's at least two characters), both if I had removed the first or not
        # Check whether the last character is special
        elif w[-1] in specialchars:
            res.append(w[:-1])
            res.append(w[-1])
        # or not
        else:
            res.append(w)
        
    # Return the list of tokens
    return res

For each word (in each line) in the documents, if either the last or the first character are specialchars, then split in multiple tokens.

In [44]:
texts_names = ['fil_1']
corpus = []
corpus_lens = []

# For each document in the directory
for text in texts_names:

    # Init a temp empty list for the words in the current document
    words = []

    # Open the document
    with open(corpus_directory + '/' + text, 'r', errors='ignore') as file:

        # For each line
        for line in file:
            # For each word in the line
            for word in line.split():
                # Tokenize it
                aux = token(word)
                # And add each token to the list of words
                for t in aux:
                    words.append(t.lower())
    # Then append the list of tokens for the document in the corpus list
    corpus.append(words)
    corpus_lens.append(len(words))

corpus_len = len(corpus)

This way, the corpus is a list of documents, which are lists of tokens.

In [45]:
# And compute the overall number of words
num_of_words_in_corpus = sum(corpus_lens)

Define a function to get a list of words from a string with words separated with a space.

In [46]:
# Given a list of strings, it returns a string
# in which the substrings will be separated by ' '
def list_to_str(strings):
    # Init res as an empty string
    res = ""
    # For each character/string in the list
    for i in range(len(strings)):
        # Concatenate the string plus a space to res
        res += strings[i] + ' '
    # Then return everything besides the last space
    return res[:-1]

Define a function to do the opposite.

In [47]:
# Given a string which contains substrings separated by ' ',
# it returns a list of words
def str_to_list(s):
    # Init res as an empty list
    res = []
    # Split the string by ' ' and for each substring
    for word in s.split():
        # Append the substring to res
        res.append(word)
    # Return the list of substrings
    return res

Declare a function which returns a list of n dictionaries, one for each n up to a fixed max, which contain the information about the frequencies for each n-gram.

In [48]:
# Create a list of n dictionaries, one for all the possible n-grams, with n in [1, max]
# Each n-dictionary will map each n-gram to a list with the absolute frequency in [0]
# and then the index of the documents in which it was found once at least,
# followed by the relative frequency (e.g. [25, 1, 20, 2, 1, 3, 4])
def create_list_of_dict_global(max):
    # Init an empty list of dictionaries
    list_dict=[]

    # For each n in [1, max], append an empty dictionary to the list
    for n in range(max):
        list_dict.append({})

    # For each index of a document in the corpus
    for i in range(corpus_len):
        # For each index of a token in the document
        for t in range(len(corpus[i])):
            # For each n in [1, max]
            for n in range(1, len(list_dict)+1):
                # If the document is not over (there is still space for an n-gram)
                if ( t + n ) <= len(corpus[i]) :
                    # If the n-gram is not yet in the n-dictionary
                    if not ( list_to_str(corpus[i][t : t+n]) in list_dict[n-1].keys() ) :
                        # Associate to the new n-gram an empty list
                        list_dict[n-1][list_to_str(corpus[i][t : t+n])] = []
                        # Set the frequency to 1 in position [0]
                        list_dict[n-1][list_to_str(corpus[i][t : t+n])].append(1)
                        # And then append the index of the current document
                        list_dict[n-1][list_to_str(corpus[i][t : t+n])].append(i)
                        # and set the relative frequency to 1
                        list_dict[n-1][list_to_str(corpus[i][t : t+n])].append(1)
                    else:
                        # Add one to the frequency of the n-gram
                        list_dict[n-1][list_to_str(corpus[i][t : t+n])][0] += 1
                        # And if the last document in which this n-gram was found
                        # is still the current one 
                        if list_dict[n-1][list_to_str(corpus[i][t : t+n])][-2] == i :
                            # Just increment the relative frequency
                            list_dict[n-1][list_to_str(corpus[i][t : t+n])][-1] += 1
                        # Otherwise
                        else:
                            # Append the new (current) document
                            list_dict[n-1][list_to_str(corpus[i][t : t+n])].append(i)
                            # and set the relative frequency to 1
                            list_dict[n-1][list_to_str(corpus[i][t : t+n])].append(1)
    # Then return the list of dictionaries
    return list_dict

Pick a max value for n. Remember that if you care about the 7-grams you need the 8-grams to be computed.

In [49]:
max_n = 8

Create a dictionary with the above function.

In [50]:
list_of_ngrams_info_dictionaries = create_list_of_dict_global(max_n)

# **STOP WORDS**

Define the function to find the stop words.

In [51]:
# Returns the extracted stop words
def get_stop_words(param=4, d=1):

    # Initialize a dictionary for sets
    neighbour_counts = defaultdict(set)

    # For each document in the corpus
    for doc in corpus:
        # For each word in the document
        for idx in range(len(doc)):
            # If the word is not the first word, add the previous word to its set of neighbours
            if idx > 0:
                neighbour_counts[doc[idx]].add(doc[idx - 1])
            # If the word is not the last word, add the next word to its set of neighbours
            if idx < len(doc) - 1:
                neighbour_counts[doc[idx]].add(doc[idx + 1])

    # Convert the sets of neighbours to counts of unique neighbours
    for word, neighbours in neighbour_counts.items():
        neighbour_counts[word] = len(neighbours)

    # Sort the words by their counts of unique neighbours
    neighbour_counts = sorted(neighbour_counts.items(), key=lambda item: item[1], reverse=True)

    # Remove from the neighbour_counts list the words that are special chars
    for special in specialchars:
        neighbour_counts = [tuple for tuple in neighbour_counts if tuple[0] != special]
    
    # Now look for the elbow point in the list
    elbow_point_index = 0
    # Iterate over the list (besides last word) to find the elbow point
    for index in range(len(neighbour_counts) - 1 - param):
        # Get the difference between the count for the current word and the next
        if abs(neighbour_counts[index][1] - neighbour_counts[index + param][1]) * d < param:
            elbow_point_index = index - 1
            break
    
    # Get the couples corresponding to all the words up to the elbow (highest num of neighbours)
    stop_word_counts = neighbour_counts[:elbow_point_index+1]
    # Get the stop words
    stop_words = [tuple[0] for tuple in stop_word_counts]
    
    # Return the stop words and the filtered expressions
    return stop_words

And compute them.

In [52]:
# Get stop words
stop_words = get_stop_words()
del corpus

# **EXPLORING GLUES**

Define a function which computes the glues.

In [53]:
# if n_max is 8, compute glues for uni-grams up to 7-grams
def compute_glues(gluename):
    # Initialize an empty list for storing the glue dictionaries
    glue_dicts = []

    # If n_max is 8, for each n in [1, 8)
    for n in range(1, len(list_of_ngrams_info_dictionaries)):

        # Initialize the glue dictionary as a dictionary with the same keys as list_of_ngrams_info_dictionaries[n-1] associating 0 to each
        glue_dict = dict.fromkeys(list_of_ngrams_info_dictionaries[n - 1], 0)

        # For each n-gram with its frequencies list
        for key, value in list_of_ngrams_info_dictionaries[n - 1].items():
            # Only if n is at least 2 or if it is a monogram of length 3
            if ( n != 1 ) or ( len(key) >= 3 ):
                # Store the absolute frequency for the n-gram
                abs_freq = value[0]

                # If n is at least 2
                if n != 1:
                    # Convert the n-gram into a list instead of a string
                    key_list = str_to_list(key)

                    # Initialize the sum to zero
                    s = 0
                    
                    # Compute the right GLUE coefficient

                    # Do Dice
                    if gluename == 'Dice':

                        # Dividing the n-gram into two parts w1...wi and wi+1...wn
                        for i in range(len(key_list) - 1):
                            # Get the absolute frequencies of the two sub-n-grams
                            f1 = list_of_ngrams_info_dictionaries[i][list_to_str(key_list[:i+1])][0]
                            f2 = list_of_ngrams_info_dictionaries[n-i-2][list_to_str(key_list[i+1:])][0]
                            # And add to the sum the partial sum
                            s += (f1 + f2) / (n - 1)

                        gl = (2 * abs_freq) / s

                    # Do SCP
                    elif gluename == 'SCP':

                        # Dividing the n-gram into two parts w1...wi and wi+1...wn
                        for i in range(len(key_list) - 1):
                            # Get the absolute frequencies of the two sub-n-grams
                            f1 = list_of_ngrams_info_dictionaries[i][list_to_str(key_list[:i+1])][0]
                            f2 = list_of_ngrams_info_dictionaries[n-i-2][list_to_str(key_list[i+1:])][0]
                            # And add to the sum the partial sum
                            s += (f1 * f2) / (n - 1)

                        gl = (abs_freq**2) / s

                    # Compute MI
                    elif gluename == 'MI':

                        # Dividing the n-gram into two parts w1...wi and wi+1...wn
                        for i in range(len(key_list) - 1):
                            # Get the absolute frequencies of the two sub-n-grams
                            f1 = list_of_ngrams_info_dictionaries[i][list_to_str(key_list[:i+1])][0]
                            f2 = list_of_ngrams_info_dictionaries[n-i-2][list_to_str(key_list[i+1:])][0]
                            # And add to the sum the partial sum
                            s += (f1 * f2) / (n - 1)

                        gl = math.log(abs_freq * num_of_words_in_corpus / s)

                    else:
                        gl = 0

                    # Add the glue to the list of the current n-gram
                    glue_dict[key] = gl

            # Otherwise if we have a monogram which is not long enough (3 letters at least)
            else:
                # Remove the list related to such n-gram from the glue dictionary
                glue_dict.pop(key)

        # Append the n-th glue dictionary to the list of glue dictionaries
        glue_dicts.append(glue_dict)

    # Return the list of glue dictionaries
    return glue_dicts

Compute the glues for each n-gram.

In [54]:
print("Computing GLUEs with SCP...")
SCP_glues = compute_glues('SCP')
print("Computing GLUEs with Dice...")
dice_glues = compute_glues('Dice')
print("Computing GLUEs with MI...")
MI_glues = compute_glues('MI')

Computing GLUEs with SCP...
Computing GLUEs with Dice...
Computing GLUEs with MI...


# **REGULAR EXPRESSIONS**

Compute a list with n dictionaries, each associating to each n-gram all the (n+1)-grams which have one more word on the left/right.

In [55]:
# List of dictionaries, one for each value of n
fathers = []

# Given max_n = 8, notice n in [1, 8) obviously, seaching among those the fathers of the children for [0, 7) that are from uni-grams to 7-grams
for n in range(1, max_n):

    # Get a dictionary with all the keys of the n-th dictionary in list_of_ngrams_info_dictionaries, and empty lists as values
    f = {key: [] for key in dict(list_of_ngrams_info_dictionaries[n - 1]).keys()}

    # For each (n+1)-gram (dictionary when n=2 actually contains 3-grams)
    for key, value in list_of_ngrams_info_dictionaries[n].items():
        # Get the (n+1)-gram as list of words
        key_list = str_to_list(key)
        # Get the two sub-n-grams of the (n+1)-gram
        subkey1 = list_to_str(key_list[1:])
        subkey2 = list_to_str(key_list[:-1])
        # Add them in the temp dictionary
        f[subkey1].append(key)
        f[subkey2].append(key)
        
    # And finally append the dictionary to the list fathers
    fathers.append(f)

Now define the function which returns a dictionary containing all the Multiword Expressions (REs).

In [56]:
# Auxiliary function (for readability) which checks whether the "key_string" n-gram is a RE
def check_MWE(key_string, glues, REglues):
    # Get n as the number of spaces in key_string + 1
    n = key_string.count(' ')
    
    # Get the glue for the n-gram
    glue = glues[n][key_string]

    # Get key_string as list to easily remove the first/last word
    key_list = str_to_list(key_string)

    # Get the set of glues for (n-1)-grams
    omega_n_minus = set()
    # If it's a 2-gram do not check for the glues of 1-grams
    if n > 1:
        omega_n_minus.add(glues[n - 1][list_to_str(key_list[1:])])
        omega_n_minus.add(glues[n - 1][list_to_str(key_list[:-1])])

    # Get the set of glues associated to all the super-n-grams
    omega_n_plus = set([glues[n + 1][fath] for fath in fathers[n][key_string]])

    # Get the max glue among all the sub-grams and super-grams
    max_glue = max(omega_n_minus.union(omega_n_plus))
    # And if it's better, store it as a new RE, associated with its glue
    if glue > max_glue:
        REglues[key_string] = glues[n][key_string]

# Returns two dictionaries, only containing REs as keys
def find_RE(glues):
    # Init the new dictionary containing only the REs mapped to their glue
    REglues = {}

    # For each n in [1, max_n)
    for n in range(1, len(glues)-1):
        # For each n-gram
        for key, _ in glues[n].items() :
            # Process the n-gram to decide whether it is a RE
            check_MWE(key, glues, REglues)

    # Return the two dictionaries
    return REglues

Then compute the REs information for each glue.

In [57]:
print("Finding REs with SCP...")
RE_SCP_glues = find_RE(SCP_glues)
print("Finding REs with Dice...")
RE_dice_glues = find_RE(dice_glues)
print("Finding REs with MI...")
RE_MI_glues = find_RE(MI_glues)

del SCP_glues
del dice_glues
del MI_glues
del fathers

Finding REs with SCP...
Finding REs with Dice...
Finding REs with MI...


# **FILTERING**

Define a function to delete REs containing special characters.

In [58]:
# Gets a string and returns false if must be deleted
def no_special(key_string):
    for i in range(len(key_string)):
        if (key_string[i] in specialchars):
            return False
    return True

Define a function to delete REs contained in one only document.

In [59]:
# Gets a string and returns false if must be deleted
def more_documents(key_list):
    if (list_of_ngrams_info_dictionaries[len(str_to_list(key_list)) - 1][key_list][0] > 1):
        return True
    else:
        return False

And then filter.

In [60]:
# Remember REs datastructures are now just dictionaries, not lists of dictionaries
RE_SCP_glues_filtered = {}
RE_dice_glues_filtered = {}
RE_MI_glues_filtered = {}

# Iterate through REs and filter SCP REs
for key, value in RE_SCP_glues.items():
    if no_special(key) and more_documents(key) :
        RE_SCP_glues_filtered[key] = value

# Iterate through REs and filter Dice REs
for key, value in RE_dice_glues.items():
    if no_special(key) and more_documents(key) :
        RE_dice_glues_filtered[key] = value

# Iterate through REs and filter MI REs
for key, value in RE_MI_glues.items():
    if no_special(key) and more_documents(key) :
        RE_MI_glues_filtered[key] = value
        
del RE_SCP_glues
del RE_dice_glues
del RE_MI_glues

# All the REs in the corresponding list of n-grams if the first and the last word are not stop-words
RE_SCP_glues_filtered = {re: glue for re, glue in RE_SCP_glues_filtered.items() if str_to_list(re)[0] not in stop_words and str_to_list(re)[-1] not in stop_words}
RE_dice_glues_filtered = {re: glue for re, glue in RE_dice_glues_filtered.items() if str_to_list(re)[0] not in stop_words and str_to_list(re)[-1] not in stop_words}
RE_MI_glues_filtered = {re: glue for re, glue in RE_MI_glues_filtered.items() if str_to_list(re)[0] not in stop_words and str_to_list(re)[-1] not in stop_words}

del stop_words

Finally get the REs for each doc.

In [61]:
def get_per_doc(REs_glues):

    #### Initialize dictionaries for unigrams and n-grams matches
    REs_per_doc = {}

    # Init lists for each word
    for k in range(corpus_len):
        REs_per_doc['doc' + str(k)] = []

    #### Populate the REs match dictionary
    for key, _ in REs_glues.items():
        n = key.count(' ')
        for index in range(1, len(list_of_ngrams_info_dictionaries[n][key]), 2):
            REs_per_doc['doc' + str(list_of_ngrams_info_dictionaries[n][key][index])].append(key)

    return REs_per_doc

Execute.

In [63]:
REs_per_doc_SCP = get_per_doc(RE_SCP_glues_filtered)
REs_per_doc_dice = get_per_doc(RE_dice_glues_filtered)
REs_per_doc_MI = get_per_doc(RE_MI_glues_filtered)

# Print
with open('../Output/REs_per_doc.txt', 'w') as f:

    # Print using SCP
    for key in REs_per_doc_SCP.keys():


        index = int(key.lstrip('doc'))
        f.write(texts_names[index])
        f.write(': ')
        f.write('\n')
        f.write('SCP - ')
        for v in REs_per_doc_SCP[key]:
            f.write(v)
            f.write('; ')
        f.write('\n')

        f.write('Dice - ')
        for v in REs_per_doc_dice[key]:
            f.write(v)
            f.write('; ')
        f.write('\n')

        f.write('Mi - ')
        for v in REs_per_doc_MI[key]:
            f.write(v)
            f.write('; ')
        f.write('\n')

    f.write('\n')